In [611]:
import numpy as np

np.random.seed(0)

## Problem

In [621]:
X = np.random.rand(1000, 2)
y = 50 * np.exp(np.sin(X)) # too difficult?
#y = 15 * np.sin(X)

In [622]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [623]:
X_train.shape, y_train.shape

((670, 2), (670, 2))

In [624]:
in_size = 2
hid_1_size = 20
hid_2_size = 15
hid_3_size = 5
out_size = 2

sizes = [in_size, hid_1_size, hid_2_size, hid_3_size, out_size]

# in_size = 2
# hid_2_size = 5
# hid_3_size = 3
# out_size = 2

# sizes = [in_size, hid_2_size, hid_3_size, out_size]

In [625]:
W = []

for i in range(1, len(sizes)):
    W.append(np.random.rand(sizes[i], sizes[i-1]))
    print(f"W_l{i} | Layer {i}, with index {i-1}")
    print(W[i-1])
    print("\n")


W_l1 | Layer 1, with index 0
[[0.03921054 0.02523183]
 [0.27155029 0.46185344]
 [0.72624328 0.4748717 ]
 [0.90405082 0.0352198 ]
 [0.18066062 0.33851449]
 [0.57749619 0.85273616]
 [0.35020195 0.26798868]
 [0.06188917 0.82130348]
 [0.37966644 0.5715502 ]
 [0.98355542 0.00159457]
 [0.14545014 0.77911099]
 [0.80512749 0.76924712]
 [0.53699889 0.97885698]
 [0.39618456 0.6019437 ]
 [0.063369   0.40985745]
 [0.72250009 0.23873884]
 [0.94382759 0.68678337]
 [0.28757538 0.76899892]
 [0.08316477 0.97477442]
 [0.04928526 0.93345589]]


W_l2 | Layer 2, with index 1
[[2.52853878e-01 7.57824108e-01 7.36994331e-05 2.54240089e-01
  7.49100607e-01 5.32336071e-01 1.14952150e-01 3.93629746e-01
  3.75549355e-01 5.68162244e-01 6.67977072e-01 8.40830242e-01
  4.97231397e-01 3.92021717e-01 1.43976534e-01 8.04822965e-01
  7.13370405e-01 4.08677397e-01 5.18432310e-01 6.65182844e-01]
 [1.64805590e-01 2.71977944e-02 3.17503699e-01 5.95585020e-01
  4.86606091e-01 6.92554627e-01 8.19689806e-01 4.88442467e-01
  1.

In [626]:
z = []
a = []


for i in range(1, len(sizes)):
    z.append(np.zeros(sizes[i]))
    a.append(np.zeros(sizes[i]))
    print(f"z_l{i} | Layer {i}, with index {i-1}:")
    print(z[i-1])
    print(f"a_l{i} | Layer {i}, with index {i-1}:")
    print(a[i-1])
    print("\n")

z_l1 | Layer 1, with index 0:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
a_l1 | Layer 1, with index 0:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


z_l2 | Layer 2, with index 1:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
a_l2 | Layer 2, with index 1:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


z_l3 | Layer 3, with index 2:
[0. 0. 0. 0. 0.]
a_l3 | Layer 3, with index 2:
[0. 0. 0. 0. 0.]


z_l4 | Layer 4, with index 3:
[0. 0.]
a_l4 | Layer 4, with index 3:
[0. 0.]




In [627]:
def forward(xi, log=False):

    try:
        
        # Input
        if log: print("Input x:")
        x = np.array([xi]).T
        if log: print(x, "\n")

        z[0] = W[0] @ x
        a[0] = 1 / (1 + np.exp(-z[0]))
        if log: print(f"W_l{1}", "\n", W[0], "\n")
        if log: print(f"z_l{1}", "\n", z[0], "\n")
        if log: print(f"a_l{1}", "\n", a[0], "\n")

        # Hidden layers, and output
        for i in range(1, len(sizes)-1):

            if log: print(f"Layer {i+1}:")
            z[i] = W[i] @ a[i-1]

            if i != len(sizes)-2:
                a[i] = 1 / (1 + np.exp(-z[i]))
                if log: print(f"W_l{i+1}", "\n", W[i], "\n")
                if log: print(f"z_l{i+1}", "\n", z[i], "\n")
                if log: print(f"a_l{i+1}", "\n", a[i], "\n")
            else:
                a[i] = z[i]
                if log: print(f"W_l{i+1}", "\n", W[i], "\n")
                if log: print(f"z_l{i+1}", "\n", z[i], "\n")
                if log: print("y", "\n", a[i], "\n")                

        return a[len(sizes)-2]

    except Exception as e:
        print(e)


def loss_MSE(x, t, log=False):
    # MSE
    if log: print("t" "\n", t, "\n")

    try:
        MSE = 0.5 * ( (t[0] - x[0])** 2 + (t[1] - x[1])** 2 )
        return MSE
    except Exception as e:
        print(e)


def backward(t):

    # Initialization
    W_grad = []
    for w in W:
        W_grad.append(np.zeros_like(w))

    delta = []
    for i in range(1, len(sizes)):
        delta.append(np.zeros(sizes[i]))

    # -------------------------------
    # Final LAYER
    # --> compute delta
    out_size = sizes[-1]
    hid_last_size = sizes[-2]
    L = len(sizes)-2
    for i in range(out_size):
        # delta^L_i = y_i - t_1
        delta[L][i] = a[L][i] - t[i]

    # --> compute W_grad
    for i in range(out_size):
        for j in range(hid_last_size):
            W_grad[L][i,j] = delta[L][i] * a[L-1][j]

    # -------------------------------
    for l in range(L, 0, -1):
        # LAYER 1
        # --> compute delta
        next_size = sizes[l+1]
        prev_size = sizes[l]
        for k in range(prev_size):
            acc = 0
            for i in range(next_size):
                acc += delta[l][i] * W[l][i,k] * a[l-1][k] * (1 - a[l-1][k])
            delta[l-1][k] = acc

        # --> compute W_grad
        pre_prev_size = sizes[l-1]
        for i in range(prev_size):
            for j in range(pre_prev_size):
                if l!=1:
                    W_grad[l-1][i,j] = delta[l-1][i] * a[l-2][j]
                else:
                    W_grad[l-1][i,j] = delta[l-1][i] * x[j]

    # UPDATE WEIGHTS
    mu = 0.00001
    for idx, w_grad in enumerate(W_grad):
        W[idx] = W[idx] - mu * w_grad

In [628]:
EPOCHS = 200
for ep in range(EPOCHS):
    for i in range(X_train.shape[0]):
        target = y_train[i]
        out = forward(X_train[i], False)
        loss = loss_MSE(out, target)
        backward(target)

    if ep%10==0:
        loss = 0
        for j in range(X_train.shape[0]):
            target = y_train[j]
            out = forward(X_train[j])
            loss += loss_MSE(out, target)
        l_train = loss / X_train.shape[0]

        loss = 0
        for j in range(X_test.shape[0]):
            target = y_test[j]
            out = forward(X_test[j])
            loss += loss_MSE(out, target)
        l_test = loss / X_test.shape[0]

        print(f"Epoch: {ep}, \t Test loss: {l_test}, \t Train loss: {l_train}")

Epoch: 0, 	 Test loss: [6183.55315267], 	 Train loss: [6272.83229318]
Epoch: 10, 	 Test loss: [3341.81627543], 	 Train loss: [3414.39899016]
Epoch: 20, 	 Test loss: [1888.60341109], 	 Train loss: [1949.22778941]
Epoch: 30, 	 Test loss: [1147.04318359], 	 Train loss: [1199.11044133]
Epoch: 40, 	 Test loss: [769.23671913], 	 Train loss: [815.18183597]
Epoch: 50, 	 Test loss: [577.131988], 	 Train loss: [618.69733353]
Epoch: 60, 	 Test loss: [479.71462684], 	 Train loss: [518.1467785]
Epoch: 70, 	 Test loss: [430.50111281], 	 Train loss: [466.69187822]
Epoch: 80, 	 Test loss: [405.77409905], 	 Train loss: [440.36146233]
Epoch: 90, 	 Test loss: [393.44769844], 	 Train loss: [426.88805504]
Epoch: 100, 	 Test loss: [387.3740121], 	 Train loss: [419.99385096]
Epoch: 110, 	 Test loss: [384.43341325], 	 Train loss: [416.4662914]
Epoch: 120, 	 Test loss: [383.04844075], 	 Train loss: [414.66143483]
Epoch: 130, 	 Test loss: [382.42542402], 	 Train loss: [413.73805309]
Epoch: 140, 	 Test loss: [38

In [629]:
# Prediction
for i in range(5):
    out = forward(X_test[i])
    loss = loss_MSE(out, y_test[i])
    print(f"input \t{X_test[i][0]} {X_test[i][1]}")
    print(f"target \t{y_test[i][0]} {y_test[i][1]}")
    print(f"out \t{out[0][0]} {out[1][0]}")
    print(f"loss \t{loss} \n")

input 	0.23277267218111208 0.3106292182913063
target 	62.972574544472614 67.87588434172426
out 	82.23956058130544 81.33990522338149
loss 	[276.24830462] 

input 	0.8020685238956139 0.37441398387974
target 	102.59802828619908 72.07809465432801
out 	82.23976223274947 81.34010379491708
loss 	[250.12190501] 

input 	0.9003674516042256 0.9832748650501553
target 	109.46208939593427 114.93193607177265
out 	82.239880562235 81.34022029193879
loss 	[934.72601141] 

input 	0.40044853473592334 0.6658713983098399
target 	73.8365920824816 92.73700851416619
out 	82.23975694868992 81.34009857067845
loss 	[100.25136801] 

input 	0.47938454938918806 0.9148630878333829
target 	79.3014324358031 110.44328122822978
out 	82.23982596537022 81.3401665260223
loss 	[427.81272095] 

